In [1]:
//hdfs dfs -mkdir -p /apps/hudi/lib
//hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar /apps/hudi/lib/hudi-spark-bundle.jar
//hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar /apps/hudi/lib/spark-avro.jar

2+2

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1610948431967_0002,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res5: Int = 4


In [2]:
%%configure -f
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1610948431967_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1610948431967_0003,spark,idle,Link,Link,✔


In [3]:
//Initialize a Spark Session for Hudi

import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor


//Where to Store Your Hudi Table 
val outputBucket = "emrpysparkhudi"
// input data set 
val inputDataBucket = "bdmtest0909"
//Specify common DataSourceWriteOptions int a single hudiOption variable 
val hudiTableName = "indian_food_review_cow"
val hudiTableRecordKey = "foodid"
val hudiTablePath = "s3://"+outputBucket+"/createdataset/"+hudiTableName
val hudiTablePartitionColumn = "diet"
val hudiTablePrecombineKey = "timestamp"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
outputBucket: String = emrpysparkhudi
inputDataBucket: String = bdmtest0909
hudiTableName: String = indian_food_review_cow
hudiTableRecordKey: String = foodid
hudiTablePath: String = s3://emrpysparkhudi/createdataset/indian_food_review_cow
hudiTablePartitionColumn: String = diet
hudiTablePrecombineKey: String = timestamp


In [4]:
// read data from s3 
val sourceData = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/*")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())
sourceData.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sourceData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [foodid: string, name: string ... 9 more fields]
root
 |-- foodid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: string (nullable = true)
 |-- cook_time: string (nullable = true)
 |-- flavor_profile: string (nullable = true)
 |-- course: string (nullable = true)
 |-- state: string (nullable = true)
 |-- region: string (nullable = true)
 |-- timestamp: long (nullable = false)



In [5]:
sourceData.select("foodid","name","diet","state","flavor_profile","course","state","timestamp").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+----------+-------------+--------------+-------+-------------+----------+
|foodid|          name|      diet|        state|flavor_profile| course|        state| timestamp|
+------+--------------+----------+-------------+--------------+-------+-------------+----------+
|     1|    Balu shahi|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1610948780|
|     2|        Boondi|vegetarian|    Rajasthan|         sweet|dessert|    Rajasthan|1610948780|
|     3|Gajar ka halwa|vegetarian|       Punjab|         sweet|dessert|       Punjab|1610948780|
|     4|        Ghevar|vegetarian|    Rajasthan|         sweet|dessert|    Rajasthan|1610948780|
|     5|   Gulab jamun|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1610948780|
|     6|        Imarti|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1610948780|
|     7|        Jalebi|vegetarian|Uttar Pradesh|         sweet|dessert|Uttar Pradesh|1610948780|
|     8|    Kaju katli|vegetar

In [6]:
// Set up our Hudi Data Source Options
val hudiOptions = Map[String,String](
    HoodieWriteConfig.TABLE_NAME -> hudiTableName,
    //for this detaset use COPY_ON_WRITE storage strategy other option us MERGE_ON_READ
    DataSourceWriteOptions.STORAGE_TYPE_OPT_KEY -> "COPY_ON_WRITE", 
    //next three options configure what Hudi should use as its record key 
    DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "foodid",
    DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> "diet",
    DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "timestamp",
    //for this data set, we specify that we want to sync metadata with Hive 
    DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true",
    DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName,
    DataSourceWriteOptions.HIVE_PARTITION_FIELDS_OPT_KEY -> "diet",
    DataSourceWriteOptions.HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY -> classOf[MultiPartKeysValueExtractor].getName
    )

//getName,
//    "hoodie.parquet.max.file.size" -> String.valueOf(1024 * 1024 * 1024),
//    "hoodie.parquet.small.file.limit" -> String.valueOf(64 * 1024 * 1024),
//    "hoodie.parquet.compression.ratio" -> String.valueOf(0.5),
//    "hoodie.insert.shuffle.parallelism" -> String.valueOf(2))


// Write input data to Hudi 

(sourceData.write
  .format("org.apache.hudi")
 // Opreation  Key tells Hudi whether this is an Insert Upsert or Bulk Insert operation 
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL)
  .options(hudiOptions)
  .mode(SaveMode.Overwrite)
  .save(hudiTablePath)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> timestamp, hoodie.datasource.hive_sync.partition_fields -> diet, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.datasource.hive_sync.table -> indian_food_review_cow, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> foodid, hoodie.table.name -> indian_food_review_cow, hoodie.datasource.write.storage.type -> COPY_ON_WRITE, hoodie.datasource.write.partitionpath.field -> diet)


In [7]:
//hive

// Show create tabel from Hive metastore 
spark.sql("SHOW TABLES").collect.foreach(println)
//show create table indian_food_review_cow;
//show partitions  indian_food_review_cow;

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[default,indian_food_review_cow,false]


In [11]:
// read Hudi data set from s3 

val readOptimzedHudiViewDF = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

// take a look at our data ... 
readOptimzedHudiViewDF.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [12]:
// select row and update - India 
val upsertdf = (readOptimzedHudiViewDF.filter($"state" === -1).withColumn("state",lit("India")))

(upsertdf.select(upsertdf("state")).distinct).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

upsertdf: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+-----+
|state|
+-----+
|India|
+-----+



In [13]:
// IMP - Before , if you wanted to update data in s3 ,, you have to read the old data , merge with the new data
// and then overwrite the old data .. with Hudi , we can directly update the data in-Place

(upsertdf.write
 .format("org.apache.hudi")
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
 .options(hudiOptions)
 .mode(SaveMode.Append)
 .save(hudiTablePath)
)


// read OptimizedHudiViewDF from updated data set 

val readOptimzedHudiViewDF_updated = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

readOptimzedHudiViewDF_updated.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF_updated: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|          India|      24|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [14]:
// delete operation 

val deleteRowDF = readOptimzedHudiViewDF_updated.filter($"state" === "Goa");
deleteRowDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

deleteRowDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------+--------------------+--------------+---------+---------+--------------+-----------+-----+------+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|foodid|    name|         ingredients|          diet|prep_time|cook_time|flavor_profile|     course|state|region| timestamp|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------+--------------------+--------------+---------+---------+--------------+-----------+-----+------+----------+
|     20210118054701|20210118054701_0_252|               252|            vegetarian|839642a3-a59b-4d6...|   252| Bebinca|Coconut milk, egg...|    vegetarian|       

In [15]:
(deleteRowDF.write
 .format("org.apache.hudi")
 .options(hudiOptions)
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
 .option(DataSourceWriteOptions.PAYLOAD_CLASS_OPT_KEY, "org.apache.hudi.common.model.EmptyHoodieRecordPayload")
 .mode(SaveMode.Append)
 .save(hudiTablePath))

//You can also hard delete data by setting OPERATION_OPT_KEY to DELETE_OPERATION_OPT_VAL to
//remove all records in the dataset you submit. For instructions on performing soft deletes, 
//and for more information about deleting data stored in Hudi tables, see Deletes in the Apache Hudi documentation.

// read OptimizedHudiViewDF from updated data set 
val readOptimzedHudiViewDF_postdelete = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

readOptimzedHudiViewDF_postdelete.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF_postdelete: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|        Gujarat|      35|
|        Haryana|       1|
|          India|      24|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
|      Telangana|       5|
+---------------+--------+
only showing top 20 rows



In [16]:
// Point in time SQL 

val commits = (spark.sql(""" select distinct(_hoodie_commit_time) as commitTime from 
indian_food_ro_table order by commitTime""").map(k=> k.getString(0)).take(50))

//commits.toString()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commits: Array[String] = Array(20210118054701, 20210118055624)


In [17]:
import org.apache.hudi.DataSourceReadOptions
val beginTime = "0"

val endTime = commits(0)

val dataSet = (spark.read
     .format("org.apache.hudi")
     // Mark that we want to do an incremental query 
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginTime) 
     .option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)  
     .options(hudiOptions)
     .load(hudiTablePath)) 

dataSet.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.hudi.DataSourceReadOptions
beginTime: String = 0
endTime: String = 20210118054701
dataSet: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [18]:
val beginTime = commits(0)

val endTime = commits(1)

val dataSet1 = (spark.read
     .format("org.apache.hudi")
     // Mark that we want to do an incremental query 
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginTime) 
     .option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)  
     .options(hudiOptions)
     .load(hudiTablePath)) 

dataSet1.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

beginTime: String = 20210118054701
endTime: String = 20210118055624
dataSet1: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+-----+--------+
|state|count(1)|
+-----+--------+
|India|      24|
+-----+--------+



In [21]:
//Using MoR


val outputBucket = "emrpysparkhudi"
val inputDataBucket = "bdmtest0909"
val hudiTableName = "indian_food_review_mor"
val hudiTableRecordKey = "foodid"
val hudiTablePath = "s3://"+outputBucket+"/createdataset/"+hudiTableName
val hudiTablePartitionColumn = "diet"
val hudiTablePrecombineKey = "timestamp"


val hudiOptions = Map[String,String](
    HoodieWriteConfig.TABLE_NAME -> hudiTableName,
    //for this detaset use COPY_ON_WRITE storage strategy other option us MERGE_ON_READ
    DataSourceWriteOptions.STORAGE_TYPE_OPT_KEY -> "MERGE_ON_READ", 
    //next three options configure what Hudi should use as its record key 
    DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "foodid",
    DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> "diet",
    DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "timestamp",
    //for this data set, we specify that we want to sync metadata with Hive 
    DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true",
    DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName,
    DataSourceWriteOptions.HIVE_PARTITION_FIELDS_OPT_KEY -> "diet",
    DataSourceWriteOptions.HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY -> classOf[MultiPartKeysValueExtractor].getName
    )

// read data from s3 
val sourceData = (spark.read.option("header",true).csv("s3://"+inputDataBucket+"/*")
             .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                 .cache())
sourceData.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

outputBucket: String = emrpysparkhudi
inputDataBucket: String = bdmtest0909
hudiTableName: String = indian_food_review_mor
hudiTableRecordKey: String = foodid
hudiTablePath: String = s3://emrpysparkhudi/createdataset/indian_food_review_mor
hudiTablePartitionColumn: String = diet
hudiTablePrecombineKey: String = timestamp
hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> timestamp, hoodie.datasource.hive_sync.partition_fields -> diet, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.datasource.hive_sync.table -> indian_food_review_mor, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> foodid, hoodie.table.name -> indian_food_review_mor, hoodie.datasource.write.storage.type -> MERGE_ON_READ, hoodie.datasource.write.partitionpath.field -> diet)
sourceData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [foodid: string, n

In [22]:
sourceData.select("foodid","name","diet","state","flavor_profile","course","state","timestamp").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+----------+-------------+--------------+-------+-------------+----------+
|foodid|          name|      diet|        state|flavor_profile| course|        state| timestamp|
+------+--------------+----------+-------------+--------------+-------+-------------+----------+
|     1|    Balu shahi|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1610948780|
|     2|        Boondi|vegetarian|    Rajasthan|         sweet|dessert|    Rajasthan|1610948780|
|     3|Gajar ka halwa|vegetarian|       Punjab|         sweet|dessert|       Punjab|1610948780|
|     4|        Ghevar|vegetarian|    Rajasthan|         sweet|dessert|    Rajasthan|1610948780|
|     5|   Gulab jamun|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1610948780|
|     6|        Imarti|vegetarian|  West Bengal|         sweet|dessert|  West Bengal|1610948780|
|     7|        Jalebi|vegetarian|Uttar Pradesh|         sweet|dessert|Uttar Pradesh|1610948780|
|     8|    Kaju katli|vegetar

In [23]:
// Write input data to Hudi 

(sourceData.write
  .format("org.apache.hudi")
 // Opreation  Key tells Hudi whether this is an Insert Upsert or Bulk Insert operation 
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL)
  .options(hudiOptions)
  .mode(SaveMode.Overwrite)
  .save(hudiTablePath)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
spark.sql("SHOW TABLES").collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[default,indian_food_review_cow,false]
[default,indian_food_review_mor_ro,false]
[default,indian_food_review_mor_rt,false]
[,indian_food_ro_table,true]


In [28]:
spark.sql("select * from indian_food_review_mor_ro").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+--------------------+---------+---------+--------------+-----------+------------+----------+----------+--------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|foodid|                name|         ingredients|prep_time|cook_time|flavor_profile|     course|       state|    region| timestamp|          diet|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+--------------------+---------+---------+--------------+-----------+------------+----------+----------+--------------+
|     20210118061352|20210118061352_1_482|                65|        non vegetarian|ff9a034b-1dd1-456...|    65|          Maach Jhol|Fish, potol, toma...|       10|       40|         spicy|main course|       Assam|North East|1610948780|non vege

In [30]:
spark.sql("""select state , count(*) from indian_food_review_mor_ro group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [31]:
spark.sql("""select state , count(*) from indian_food_review_mor_rt group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [33]:
// read Hudi data set from s3 

val readOptimzedHudiViewDF = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

// take a look at our data ... 
readOptimzedHudiViewDF.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
|             -1|      24|
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [34]:
// select row and update - India 
val upsertdf = (readOptimzedHudiViewDF.filter($"state" === -1).withColumn("state",lit("India")))

(upsertdf.select(upsertdf("state")).distinct).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

upsertdf: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+-----+
|state|
+-----+
|India|
+-----+



In [35]:
(upsertdf.write
 .format("org.apache.hudi")
 .option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
 .options(hudiOptions)
 .mode(SaveMode.Append)
 .save(hudiTablePath)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
// read OptimizedHudiViewDF from updated data set 

val readOptimzedHudiViewDF_updated = (spark.read
                             .format("org.apache.hudi")
                             .load(hudiTablePath+ "/*"))

readOptimzedHudiViewDF_updated.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimzedHudiViewDF_updated: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 14 more fields]
+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|          India|      24|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [38]:
spark.sql("""select state , count(*) from indian_food_review_mor_rt group by 
state order by state ASC """ ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          state|count(1)|
+---------------+--------+
| Andhra Pradesh|      10|
|          Assam|      21|
|          Bihar|       3|
|   Chhattisgarh|       1|
|            Goa|       3|
|        Gujarat|      35|
|        Haryana|       1|
|          India|      24|
|Jammu & Kashmir|       2|
|      Karnataka|       6|
|         Kerala|       8|
| Madhya Pradesh|       2|
|    Maharashtra|      30|
|        Manipur|       2|
|   NCT of Delhi|       1|
|       Nagaland|       1|
|         Odisha|       7|
|         Punjab|      32|
|      Rajasthan|       6|
|     Tamil Nadu|      20|
+---------------+--------+
only showing top 20 rows



In [41]:
// Point in time SQL 

val commits = (spark.sql(""" select distinct(_hoodie_commit_time) as commitTime from 
indian_food_review_mor_rt order by commitTime""").map(k=> k.getString(0)).take(50))

commits.toString()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commits: Array[String] = Array(20210118061352, 20210118062628)
res143: String = [Ljava.lang.String;@2987aa63


In [ ]:
//val beginTime = commits(0)

//val endTime = commits(1)

val dataSet1 = (spark.read
     .format("org.apache.hudi")
     // Mark that we want to do an incremental query 
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, "20210118061352") 
     //.option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)  
     .options(hudiOptions)
     .load(hudiTablePath)) 

dataSet1.registerTempTable("indian_food_ro_table");
spark.sql("""select state , count(*) from indian_food_ro_table group by 
state order by state ASC """ ).show()

//https://github.com/apache/hudi/issues/998